In [ ]:
import datetime

from pydantic import ConfigDict
import polars as pl
from tqdm import tqdm
import matplotlib.pyplot as plt

import stock
from stock.algorithm.market import is_limit_high

In [ ]:
# 細かく利益確定していくsimulation

In [ ]:
stacked_df = stock.watchlist.v1.get_watch_list_all()

In [ ]:
class StopCondition(stock.simulation.base_condition.BaseCondition):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    # parameter
    max_loss_rate: float = 0.08
    sell_rate: float = 0.02
    max_days: int = 7 * 2
    # results
    buying_price: float = -1
    buying_date: datetime.date = datetime.date.today()
    selling_price: float = -1
    selling_date: datetime.date = datetime.date.today()
    # internal
    loss_cut_price: float = -1
    target_price: float = -1
    index: int = -1
    df: pl.DataFrame = pl.DataFrame()

    def reset_results(self):
        self.buying_price = -1
        self.buying_date = datetime.date.today()
        self.selling_price = -1
        self.selling_date = datetime.date.today()
        self.loss_cut_price = -1
        self.target_price = -1
        self.index = -1
        self.df = pl.DataFrame()

    def set_start(self, src_df: pl.DataFrame, start_date: datetime.date) -> float:
        #print(src_df)
        self.reset_results()
        df = src_df.filter(pl.col("date") >= start_date).sort(pl.col("date"))
        if len(df) < 15:
            return -1
        
        if df["date"][0] - start_date > datetime.timedelta(days=10):
            return -1
        
        if is_limit_high(df["close"][0], df["open"][1]):
            return -1
        
        self.buying_price = df["open"][1]
        self.buying_date = df["date"][1]

        self.loss_cut_price = self.buying_price * (1 - self.max_loss_rate)
        self.target_price = self.buying_price * (1 + self.sell_rate)
        self.df = df
        self.index = 1
        return self.buying_price

    def run_simulation(self) -> float:

        if self.df["date"][self.index] - self.buying_date > datetime.timedelta(days=self.max_days):
            self.selling_date = self.df["date"][self.index]
            self.selling_price = self.df["open"][self.index]

        if self.df["low"][self.index] < self.loss_cut_price:
            self.selling_date = self.df["date"][self.index]
            self.selling_price = min(self.df["open"][self.index], self.loss_cut_price)

        if self.df["high"][self.index] > self.target_price:
            self.selling_date = self.df["date"][self.index]
            self.selling_price = max(self.df["open"][self.index], self.target_price)

        self.index += 1
        return self.selling_price

In [ ]:
stacked_df

In [ ]:
# 特定の月でのsimulation
target_year = 2024
target_month = 7
date = datetime.date(target_year, target_month, 1)
code_list = stock.kabutan.get_code_list()

res_list = []
while date < datetime.date(target_year, target_month + 1, 1):
    for code in tqdm(code_list):
        condition = StopCondition(sell_rate=0.01, max_loss_rate=0.01)
        if stock.kabutan.data.calc_estimated_capitalization(code) > 100000000000:
            continue
        res = stock.simulation.simulate.run(code, date, condition)
        res_list.append(res)
    date += datetime.timedelta(days=1)

In [ ]:
res_list_per_day = [[] for _ in range(31)]
for res in res_list:
    if res.buying_price < 0 or res.buying_date.month > target_month:
        continue
    idx = res.buying_date.day - 1
    res_list_per_day[idx].append(res)

profits_per_day = {
    ress[0].buying_date: [res.profit for res in ress] for ress in res_list_per_day if len(ress) > 0
}
for date, profits in profits_per_day.items():
    if len(profits) > 0:
        print("date = {}, sum = {:>10.4f}, mean = {:>10.4f}".format(
            date, sum(profits), sum(profits) / len(profits)))
all_profits = [sum(profits) / len(profits) for _, profits in sorted(profits_per_day.items()) if len(profits) > 0]

In [ ]:
profits_per_day = {}
for res in results:
    if res.buying_price < 0:
        continue
    if res.buying_date.year == target_year and res.buying_date.month == target_month:
        target_date = res.buying_date
        if target_date not in profits_per_day:
            profits_per_day[target_date] = []
        profits_per_day[target_date].append(res.profit)

for date, profits in sorted(profits_per_day.items()):
    if len(profits) > 0:
        print("date = {}, sum = {:>10.4f}, mean = {:>10.4f}".format(
            date, sum(profits), sum(profits) / len(profits)))        
selected_profits = [sum(profits) / len(profits) for _, profits in sorted(profits_per_day.items()) if len(profits) > 0]

In [ ]:
sum(all_profits), sum(selected_profits)

In [ ]:
sum(all_profits), sum(selected_profits)

In [ ]:
sum(all_profits), sum(selected_profits)

In [ ]:
sum(all_profits), sum(selected_profits[1:])

In [ ]:
plt.plot(all_profits)
plt.plot(selected_profits)

In [ ]:
results = []
for index in tqdm(range(len(stacked_df))):
    condition = StopCondition(sell_rate=0.01, max_loss_rate=0.01)
    res = stock.simulation.simulate.run(
        stacked_df["code"][index], stacked_df["date"][index], condition
    )
    results.append(res)

In [ ]:
profits = [res.profit for res in results if res.profit < 0.02]
sum(profits), sum(profits) / len(profits)

In [ ]:
index = 21
print(stacked_df["code"][index])
print(results[index].model_dump_json(indent=4))